In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy as npt
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
tb_default = pd.read_csv('data/Default.csv', index_col=0)

tb_default['default_binary'] = tb_default['default'].apply(
    lambda x: 1 if x == "Yes" else 0)

In [ ]:
tb_default.head()

# Classification in ML

## Logistic Regression

In [ ]:
sns.scatterplot(data=tb_default,
                x='balance',
                y='income',
                hue='default',
                palette='tab10')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 8))

sns.boxplot(data=tb_default, x='default', y='balance', ax=ax[0])
sns.boxplot(data=tb_default, x='default', y='income', ax=ax[1])

In [ ]:
sns.scatterplot(data=tb_default, x='balance', y='default')

In [ ]:
lr = LinearRegression()
lr.fit(X=tb_default[['balance']], y=tb_default['default_binary'])

In [ ]:
tb_default['pred_lmfit'] = lr.predict(tb_default[['balance']])

In [ ]:
tb_default['pred_lmfit']

In [ ]:
sns.scatterplot(data=tb_default, x='balance', y='default_binary', alpha = 0.5)
sns.lineplot(data=tb_default, x='balance', y='pred_lmfit')

- Predictions may be out of range.
- But yeah, the predictions will be ordered.
- Problem is - this approach cannot be extended to qualitative responses containing more than two levels.

# Logistic Regression

Rather than predicting the `target` directly, `logistic regression` tries to model the <b>`probability`</b> that your `target` belongs to a particular category.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

sns.regplot(data=tb_default,
            x='balance',
            y='default_binary',
            color='blue',
            ax=ax[0])
sns.regplot(data=tb_default,
            x='balance',
            y='default_binary',
            logistic=True,
            color='blue',
            ci=None,
            ax=ax[1])

ax[0].set_title('Linear Regression')
ax[1].set_title('Logistic Regression')

S-shaped curve.

$$ \frac{e^{x}}{(1+e^{x})} $$

In [ ]:
x = np.arange(-10, 10, 0.13)

plt.plot(x, np.exp(2*x)/(1 + np.exp(2*x)))


## Linear Regression
If we were to use a linear regression, the equation would be:

$$ y = P(default=Yes | balance) = a_0 + a_1\cdot balance $$

## Logistic Regression
The logistic regression seeks to model the probability in a better way:

$$ P(default=Yes | balance) = \frac{e^{a_0 + a_1\cdot x}}{1 + e^{a_0 + a_1\cdot x}}$$



# In Python

In [ ]:
tb_default.head()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic = LogisticRegression()
logistic.fit(X=tb_default[['balance']], y=tb_default['default_binary'])

In [ ]:
logistic.predict_proba(tb_default[['balance']])

In [ ]:
tb_default['pred_prob'] = logistic.predict_proba(tb_default[['balance']])[:,1]

In [ ]:
tb_default['default_binary']

In [ ]:
sns.scatterplot(data = tb_default, x = 'balance', y = 'pred_prob')

Como fazemos para ter uma previsão categórica, como nossa variável resposta? Utilizamos o conceito de **threshold**: utilizamos um valor de probabilidade pelo qual dividiremos as previsões - abaixo deste valor todas as previsões serão `False` acima, `True`.

Para classificação binária esse threshold é tipicamente 0.5.

In [ ]:
tb_default['pred_binary_where'] = np.where(tb_default['pred_prob'] > 0.5, 1, 0)

In [ ]:
tb_default['pred_binary'] = logistic.predict(tb_default[['balance']])

In [ ]:
sum(tb_default['pred_binary_where'] == tb_default['pred_binary'])

In [ ]:
sns.scatterplot(data = tb_default, x = 'balance', y = 'pred_prob')
sns.scatterplot(data = tb_default, x = 'balance', y = 'pred_binary')

## Medindo o erro

In [ ]:
tb_default['erro_binario'] = np.where(
    tb_default['pred_binary'] == tb_default['default_binary'], 0, 1)

In [ ]:
sum(tb_default['erro_binario'])

In [ ]:
1-sum(tb_default['erro_binario'])/len(tb_default['erro_binario'])

In [ ]:
logistic.score(tb_default[['balance']], tb_default['default_binary'])

In [ ]:
tb_default.groupby(['default_binary'])['erro_binario'].agg(['sum', 'count'])

A **acurácia** é uma métrica simples, direta e fácil de se explicar. No entanto, muitas vezes ela não é suficiente para comparar/avaliar modelos:

* Em problemas onde o **tamanho das duas classes é muito diferente** ela oculta a taxa de erro na classe minoritaria.
* Muitas vezes o **custo** de um falso positivo e um falso negativo não são equivalentes. No exemplo acima o custo de deixar de emprestar (custo de oportunidade) pode ser muito diferente do custo de calote.

Podemos extender a avaliação de erro do modelo atrvés de **outras métricas** de erro que tratem dos diferentes problemas levantados acima. Primeiro, vamos analisar a curva ROC para entender melhor o trade-off que ocorre quando mudamos o threshold de classificação.

In [ ]:
tb_erro = tb_default.groupby(['default_binary'])['erro_binario'].agg(['sum', 'count']).reset_index()
tb_erro.columns = ['valor_verdadeiro', 'erros', 'total']
tb_erro['acertos'] = tb_erro['total'] - tb_erro['erros']
tb_erro

A *taxa de positivos verdadeiros* (TPR, recall ou sensibilidade) é **100/333**, ou seja, o número de positivos que o modelo previu **corretamente** dividido pelo número de positivos total.

A *taxa de falsos positivos* (FPR ou fall-out) é **233/333**, ou seja, o número de positivos que o modelo previu **incorretamente** dividido pelo número de positivos total.

Existe um trade-off entre TPR e FPR: conforme aumento o threshold diminuo a FPR mas aumento a TPR (e vice-versa). A forma mais simples de visualizar este trade-off é através da curva ROC.

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true = tb_default['default_binary'],
                                 y_score = tb_default['pred_prob'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
ax[0].plot(thresholds, tpr)
ax[0].set_title('Curva de TPR (Recall)')
ax[0].set_xlim([0, 1])
ax[0].set_xlabel('Threshold')
ax[0].set_ylabel('TPR')
ax[1].plot(thresholds, fpr)
ax[1].set_title('Curva de FPR')
ax[1].set_xlim([0, 1])
ax[1].set_ylabel('FPR')
fig.suptitle('Curva de FPR/TPR por Threshold')

In [ ]:
fig, ax = plt.subplots(figsize = (7,7))
ax.plot(fpr, tpr, label = 'Modelo')
ax.plot(fpr, fpr, label = 'Aleatório')
ax.set_xlabel('FPR - Taxa de Positivos Falsos')
ax.set_ylabel('TPR - Taxa de Positivos Verdadeiros (Recall)')
ax.set_aspect('equal')
plt.legend()
fig.suptitle('Curva ROC (Receiver Operating Characteristic)');

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_score = roc_auc_score(y_true=tb_default['default_binary'],
              y_score=tb_default['pred_prob'])
print(f"Área debaixo da Curva ROC: {round(roc_score, 2)}")

Embora a curva ROC represente bem o impacto que a mudança de threshold tem sobre o erro de classificação ainda temos um problema: o desbalanceamento das classes. Assim como a acurácia, a curva ROC dá peso demais para as classificações negativas corretas (classe majoritaria), ocultando o erro sobre a classificação positiva (classe minoritaria).

Para consolidar a nossa avaliação do erro de previsão, precisamos ver ainda outra métrica, que lida melhor com problemas desbalanceados - a precisão. A precisão é a taxa entre o número de positivos verdadeiros e o número de positivos previstos.

In [ ]:
tb_confusion = tb_default.groupby(['default_binary', 'pred_binary'])['pred_prob'].count().reset_index()

In [ ]:
tb_confusion.pivot_table(columns='pred_binary', index='default_binary')

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_true = tb_default['default_binary'],
                 y_pred = tb_default['pred_binary'])

In [ ]:
precision = 100/(42 + 100)
recall = 100/(100+233)
print(f"Precisão: {precision} - Recall: {recall}")

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
prc, rec, thresh = precision_recall_curve(y_true=tb_default['default_binary'],
                                          probas_pred=tb_default['pred_prob'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
ax[0].plot(thresh, prc[:-1])
ax[0].set_title('Curva de Precisão')
ax[0].set_xlim([0, 1])
ax[0].set_xlabel('Threshold')
ax[0].set_ylabel('Precisão')
ax[1].plot(thresh, rec[:-1])
ax[1].set_title('Curva de Recall (TPR)')
ax[1].set_xlim([0, 1])
ax[1].set_xlabel('Threshold')
ax[1].set_ylabel('Recall (TPR)')
fig.suptitle('Curva de FPR/TPR por Threshold')

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
ax.plot(rec, prc)
ax.set_title('Curva de R-P (Recall-Precision)')
ax.set_xlim([0, 1])
ax.set_xlabel('Recall (TPR)')
ax.set_ylabel('Precisão')

In [ ]:
f1 = 2 * (precision * recall)/(precision + recall)
print(f"F1-Score: {f1}")

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
print(f1_score(y_true = tb_default['default_binary'],
             y_pred = tb_default['pred_binary']))

### Conclusão

#### Acurácia

**PROS**

* Fácil de explicar/entender;
* Medida direta, conversa diretamente com o que as pessoas imaginam ser o *erro do modelo*.

**CONTRAS**

* Não representa bem o erro em problemas de classes desbalanceadas.

**QUANDO USAR**

* Apenas em problemas balanceados;
* Quando as classes previstas não tem custo diferente.

#### Matriz de Confusão

**PROS**

* Fácil de explicar/entender;
* Permite a visualização de todos os erros do modelo.

**CONTRAS**

* Não é um indicador;
* Não permite a avaliação automatica de modelos.

**QUANDO USAR**

* Na fase exploratória de modelagem, para comparar diferentes versões iniciais do modelo. Sempre que estamos construindo os modelos manualmente e podemos analisar o resultado de cada um particularmente.

#### AUC-ROC (e Curva ROC)

**PROS**

* A curva ROC é uma boa representação do trade-off entre precisão e falsos positivos;
* O AUC-ROC score não é específico para um threshold, permitindo uma avaliação do score (a probabilidade prevista) do modelo.

**CONTRAS**

* O AUC-ROC score não representa bem o erro em problemas de classes desbalanceadas.

**QUANDO USAR**

* Na fase exploratória de modelagem, para comparar diferentes versões iniciais do modelo. Sempre que estamos construindo os modelos manualmente e podemos analisar o resultado de cada um particularmente;
* Em problemas de classes balanceadas.

#### F1-SCORE

**PROS**

* A medida padrão dentro da área de modelagem/ciência de dados. Qualquer outro analista/cientista de dados vai entender o que você está falando;
* A utilização da precisão torna o F1 um bom método para medir o erro em problemas desbalanceados.

**CONTRAS**

* Difícil de explicar (fora de *0 é ruim 1 é bom*)

**QUANDO USAR**

* Para comparar 2 ou mais modelos quantitativamente;
* Durante métodos de seleção de variáveis/técnicas automáticos.

# Interpretando o modelo

Primeiro, assim como na regressão, vamos normalizar as variáveis de entrada para termos uma interpretação do intercepto mais natural:

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(tb_default[['balance']])
y = tb_default['default_binary']

In [ ]:
log_fit = LogisticRegression()
log_fit.fit(X, y)

## Primeiro o intercepto

In [ ]:
print(f"Intercepto: {log_fit.intercept_[0]}")

O que isso significa?? Precisamos lembrar que a regressão logística projeta o logaritmo das chances (o chamado logit)

$$ P(default) = \frac{e^{a_0 + a_1\cdot x}}{1 + e^{a_0 + a_1\cdot x}}$$


$$ \left(\frac{P(default)}{1 - P(default)}\right) = e^{a_0 + a_1\cdot x}$$

In [ ]:
np.exp(log_fit.intercept_[0])*1000

Exponenciar o intercepto nos dá a **chance** (p/(1-p), não a probabilidade p) média de default:

In [ ]:
intercepto = np.exp(log_fit.intercept_[0])*1000
print(f"Chance de default em com balance médio: {round(intercepto,2)}:1000")

## Depois os coeficientes

In [ ]:
print(f"Coeficiente Balance: {log_fit.coef_[0][0]}")

In [ ]:
np.exp(log_fit.coef_[0][0])

O coeficiente nos diz o quanto a variação de uma unidade de X impacta, multiplicativamente, a **chance** (de novo, não a probabilidade) de default. No caso acima vemos que:

In [ ]:
impacto_1x = np.exp(log_fit.coef_[0][0])
print(f"Chance de default em +1 desvio padrão de X: {round(impacto_1x * intercepto,2)}:1000")

## O Problema de Chances

O problema das interpretações acima é que, a não ser que trabalhemos em uma casa de apostas, **chances não são facilmente interpretáveis**. Uma solução é criar um gráfico mostrando o **impacto da variação de X sobre a PROBABILIDADE**!

In [ ]:
impacto_x = np.linspace(-2, 4, 100)

In [ ]:
odds = np.exp(log_fit.intercept_[0] + impacto_x * log_fit.coef_[0][0])

In [ ]:
log_fit.coef_[0][0]*2

In [ ]:
probabilities = (odds/(1+odds))

In [ ]:
tb_simul = pd.DataFrame({
    'impacto_x': impacto_x,
    'odds': odds,
    'prob': probabilities
})

In [ ]:
sns.scatterplot(data = tb_simul, x = 'impacto_x', y = 'prob')

In [ ]:
tb_default['X'] = scaler.fit_transform(tb_default[['balance']])

In [ ]:
tb_default['pred_prob'] = log_fit.predict_proba(tb_default[['X']])[:,-1]

In [ ]:
sns.scatterplot(data = tb_default, x = 'X', y = 'pred_prob')

## Modelo com mais coeficientes

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(tb_default[['balance', 'income']])
y = tb_default['default_binary']
log_fit2 = LogisticRegression()
log_fit2.fit(X, y)

In [ ]:
impacto_bal = np.linspace(-4, 4, 100)
odds = np.exp(log_fit2.intercept_[0] + impacto_bal * log_fit2.coef_[0][0])
probabilities = (odds/(1+odds))
tb_simul = pd.DataFrame({
    'impacto_bal': impacto_bal,
    'odds': odds,
    'prob': probabilities
})
sns.scatterplot(data = tb_simul, x = 'impacto_bal', y = 'prob')

In [ ]:
impacto_inc = np.linspace(-4, 4, 100)
odds = np.exp(log_fit2.intercept_[0] + impacto_inc * log_fit2.coef_[0][1])
probabilities = (odds/(1+odds))
tb_simul = pd.DataFrame({
    'impacto_inc': impacto_inc,
    'odds': odds,
    'prob': probabilities
})
sns.scatterplot(data = tb_simul, x = 'impacto_inc', y = 'prob')

# EXTRA - The Loss function of the Logistic Regression

What does the logistic regression tries to minimize? 

Intuitively, we want to assign more punishment when predicting 1 while the actual is 0 and when predict 0 while the actual is 1.

\begin{equation}
  Cost(\hat{p}, y_{obs}) =
    \begin{cases}
      -log(\hat{p}) & \text{if } y_{obs} = 1  \\
      -log(1-\hat{p}) & \text{if } y_{obs} = 0\\
    \end{cases}       
\end{equation}



$$Cost(\hat{p}, y_{obs}) = -y_{obs} \cdot log(\hat{p}) - (1 - y_{obs})\cdot log(1-\hat{p})$$

$\hat{p}$ is my estimated probability, and $y_{obs}$ is the label of my observation.

So let's understand what this `cost function` represents:
- Imagine I have an observation whose true default label is 0 ($y_{obs}$ = 0) and my model predicts that the probability of its value being 1 (default) is 80%. We would have:
    
    - $cost(0.8, 0) = -0 \cdot log(0.8) - 1 \cdot log(1-0.8) = -log(0.2) \approx 1.6$

Now if we say that the probability of it being 1 is 90%:
- $cost(0.9, 0) = -0 \cdot log(0.9) - 1 \cdot log(1-0.9) = -log(0.1) \approx 2.3$

Now if we say that the probability of it being 1 is 95%:
- $cost(0.95, 0) = -0 \cdot log(0.95) - 1 \cdot log(1-0.95) = -log(0.05) \approx 3$

In [ ]:
fig, ax = plt.subplots(1, 2, sharey=True, figsize=(12,4))
y_obs = 0
p = np.arange(0.01, 1, 0.01)
cost = -y_obs * np.log(p) - (1-y_obs) * np.log(1-p)
ax[0].plot(p, cost)

y_obs = 1
p = np.arange(0.01, 1, 0.01)
cost = -y_obs * np.log(p) - (1-y_obs) * np.log(1-p)
ax[1].plot(p, cost)

ax[0].set_title('$y_{true}$ = 0')
ax[1].set_title('$y_{true}$ = 1')

ax[0].set_ylabel('Cost')

ax[0].set_xlabel('Probability of y = 1')
ax[1].set_xlabel('Probability of y = 1')

Thus, it penalizes when you are sure it is one, but you are wrong. Or it penalizes when you are sure it is zero, but you are wrong, the true label is one.

So this is what logistic regression tries to minimize. Two important summaries:

- The results of the logistic regression are <b>probabilities</b> of being the label 1.
- As it minimizes that cost function, <b>you can be very confident of observations predicted with probabilities close to 1 or close to 0</b>. They will probably not be wrong because your model tried to avoid it during training.

## Odds - the chances

$$ P(default) = \frac{e^{a_0 + a_1\cdot x}}{1 + e^{a_0 + a_1\cdot x}}$$


$$ \left(\frac{P(default)}{1 - P(default)}\right) = e^{a_0 + a_1\cdot x}$$

1 in 5 people is a fraudster.

P = 1/5 = 0.2

Odds = $\frac{0.2}{0.8} = 1/4 = 0.25$

Odds: $\frac{\text{favorable events}}{\text{unfavorable events}}$, Probability: $\frac{\text{favorable events}}{\text{total events}}$

## Log Odds

$$ log\left(\frac{P(default)}{1 - P(default)}\right) = a_0 + a_1x$$

Remember that for **Linear Regression**, the value a_1, the coefficient, can be understood as how much of our target change if we change 1 unit in `x`. That is, if we change 1 in `x`, our target changes by $a_1$

For **Logistic Regression**, though, increasing X by one unit affects the **log odds** in $a_1$. So, although increasing `x` indeed increases the probability P, the value it will increase depends on X.
